# Attempt: Predicting moves based on Win probability

##### We attempted to create a model that would predict a move based on win probability of higher than 0.5. Essentially, we trained the model on moves that were generally associated with a final game outcome of winning.

##### The input to the model is an 8x8 chess board, with white pieces encoded in 6 channels, and black pieces encoded in another 6 channels, because there are 6 different types of pieces: pawns, knights, bishops, rooks, queens, and kings. So we have a 12 x 8 x 8 tensor as our input to the model.

##### The output of the model is a 64 x 64 tensor that is the move prediction. The rows represent the start square, of which there are 64 possibilities, and the columns represent the end square, of which there are 64 possibilities.

##### Similar to the other win probability model, we used a Convolutional Neural Network architecture. By contrast, this model uses win probability to predict the best move.

##### This ended up not working, with it making up illegal moves.

In [ ]:
!pip install chess pgnparser numpy tensorflow keras

In [26]:
#imports
import chess
import chess.pgn
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [ ]:
!wget https://database.lichess.org/standard/lichess_db_standard_rated_2024-01.pgn.zst

In [ ]:
!apt-get install zstd

In [5]:
!unzstd --stdout lichess_db_standard_rated_2024-01.pgn.zst | head -n 50000 > small_games.pgn

In [ ]:
#parsing game from pgn file
def parse_pgn(pgn_file, num_games=2000):
    games = []
    with open(pgn_file) as f:
      #for each game in the pgn
        for _ in range(num_games):
            game = chess.pgn.read_game(f)
            if game is None:
                break
            games.append(game)
    return games

#loading games
games = parse_pgn("small_games.pgn", num_games=2000)


In [18]:
import numpy as np
import chess

def fen_to_tensor(fen):
    board = chess.Board(fen)
    # 12 channels to represent board: 6 for white, 6 for black
    # for each channel, store the board state as an 8x8 grid (size of chess board)

    tensor = np.zeros((12, 8, 8), dtype=np.float32)

    #encoding white pieces as capital letters, black pieces as lower case
    piece_map = {
        "P": 0, "N": 1, "B": 2, "R": 3, "Q": 4, "K": 5,
        "p": 6, "n": 7, "b": 8, "r": 9, "q": 10, "k": 11,
    }


    # given the board state in fen notation, iterate over every square on the board that has a piece on it, to get their positions
    for square, piece in board.piece_map().items():

        # square is an integer from 0 to 63 to represent all possible positions on board, so doing these operations gets row and column
        row = 7 - (square // 8)

        col = square % 8

        #piece.symbol() is a character, like P, N, B, etc.
        color = piece_map[piece.symbol()] # defaults to white (0-5)

        # first 6 channels are white, last 6 are black (for 12 x 8 x 8 tensor)
        if piece.color == chess.BLACK:
            color += 6 # to get 6-11

        #encode piece's position in tensor
        tensor[color, row, col] = 1
    return tensor


# return the move as 2 numbers that represent the beginning and ending square
def move_to_index(move):
    return move.from_square, move.to_square

In [ ]:
def extract_labels(games, max_moves=50):

  #X is initial input board state
  #Y is the move prediction, based on win probability of > 0.5
    X, y = [], []
    for game in games:
        board = game.board()
        result = game.headers["Result"]

        # assigning 1, 0, or 0.5 win probability based on white's perspective
        if result == "1-0":
           # White wins
            win_prob = 1.0
        elif result == "0-1":
           # Black wins
            win_prob = 0.0
        else:
          # Draw
            win_prob = 0.5

        move_count = 0

        #iterating through each move in the game
        for move in game.mainline_moves():


            #adds board state to input tensor
            X.append(fen_to_tensor(board))

            #get the two integers associated with move
            fromSquare, destSquare = move_to_index(move)
            if win_prob > 0.5:


              # since the output layer is 64x64 for the move prediction, we need to append to y based on that

              # initializing with 0's
              best_move = np.zeros((64, 64), dtype=np.float32)

              #just setting the relevant from and to coordinate to 1. Everything else is 0
              best_move[fromSquare, destSquare] = 1

              y.append(best_move)


              move_count += 1
            #stops when max moves = 50 is reached
            if move_count >= max_moves:
                break

            #goes to the next move in the game
            board.push(move)

    #return collection of board states and move predictions based on win probability
    return np.array(X), np.array(y)

X, y = extract_labels(games)


In [ ]:
model = Sequential([


    #first convolutional layer: going from 12 x 8 x 8 to 64 filters (64 x 8 x 8)
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(12, 8, 8)),

    #second convolutional layer: going from input of 64 x 8 x 8 to 128 filters (128 x 8 x 8)
    Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),



    #transform to 1D vector for fully connected layers
    Flatten(),

    #1st fully connected layer, using relu. Input is 128 x 8 x 8 vector (before flatten), and output is 1D vector that has 512 elements
    Dense(512, activation='relu'),


    #2nd fully connected layer, which takes input as 1D vector with 512 elements, and outputs 64x64 tensor, which is the move prediction (since it's not probability, don't have to use relu)
    Dense(64*64, activation='linear')
])

#we're using cross entropy because this is a classification task, with multiple possible classes (64x64)
# adam is used a lot as an optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()


In [ ]:
#training the model on 10 epochs, batch size 32, train test split of 80:20
#X: board state you're given
#y: associated predicted move, given in form of 64x64 grid where rows represent start squares, columns represent destination squares
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)
